In [ ]:
# label relevancy

In [6]:
import numpy as np
from scipy.stats import spearmanr
import torch

In [2]:
# cosine similarity
def cosine(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [22]:
# load-in your training probabilities (alternatively, test probabilities or label embeddings)
prob = np.load('BGC/train_prob.npy')

In [23]:
# initialize
K = prob.shape[1]
result_re = np.zeros((K,K))

In [24]:
# calculate label relevancy (whole-hierarchy)
for i in range(K):
    for j in range(K):
        if i<j:
            # if probabilities are used
            result_re[i,j] = result_re[j,i] = spearmanr(prob[:,i],prob[:,j])[0]
            
            # if embeddings are used
            # result_re[i,j] = result_re[j,i] = cosine(prob[:,i],prob[:,j])

In [25]:
# normalize
result_re = np.abs(result_re)
result_re = result_re/np.sum(result_re,axis=1,keepdims=True)
result_re += np.eye(K)

In [26]:
np.save("BGC/adjacent_matrix.npy", result_re)

In [ ]:
# label confusion and sample importance learning

In [50]:
result_co = np.zeros((K,K))

In [34]:
# load in training set probabilities and true labels
prob = np.load('BGC/train_prob.npy') 
labels = torch.load("BGC/bgc_val_y_3.pt")[0:prob.shape[0],:]

In [56]:
# calculate label confusion
for i in range(K):
    for j in range(K):
        index = np.array(labels[:,i]*(1-labels[:,j]), dtype='bool')
        if np.sum(index) != 0: 
            result_co[i,j] = np.sum(prob[index,j])/np.sum(index)
        else:
            pass

In [78]:
result_co[result_co<0.1] = 0

In [79]:
# assign sample weights
n = len(labels)
tau = 1 # temperature parameter
weights = np.zeros((n,K))
for i in range(n):
    for j in range(K):
        if labels[i,j]==0:
            weights[i,j]=1
        else:
            index = np.array(1-labels[i,:], dtype='bool')
            temp = np.sum(np.exp(tau * result_co[j,index])-1)
            weights[i,j] = 1+temp

In [84]:
np.save("BGC/sample_weight.npy", weights)

In [92]:
torch.save(torch.cat((labels, torch.tensor(weights)), dim=1), 
          "BGC/bgc_train_labels&weights_10016.pt")